# 🧠 Employee Attrition Prediction System
## XAI-Powered Models for Managerial Decision-Making

---

**Research Basis:** Baydili & Tasci, *"Predicting Employee Attrition: XAI-Powered Models for Managerial Decision-Making,"* Systems 2025, 13, 583

### What this notebook demonstrates:
1. **Transformer Encoder** for tabular HR data prediction
2. **SHAP Explainability** — why each employee is at risk
3. **LDA Topic Modeling** — hidden themes in employee reviews
4. **Five-Tier Risk Classification** — actionable risk levels

### How to run:
1. Click **Runtime → Run all** (or press `Ctrl+F9`)
2. To use your own data: upload a CSV in the **"Upload Your Dataset"** section
3. The system auto-detects IBM HR, Kaggle HR, and AmbitionBox formats

---

## 📦 Step 0: Install Dependencies

In [ ]:
# Install required packages (most are pre-installed in Colab)
!pip install -q numpy pandas scikit-learn matplotlib seaborn
print("✅ All dependencies installed!")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
from collections import Counter
import json
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print("✅ All imports successful!")
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")

---

## 🔧 Step 1: Configuration

These are the hyperparameters that control the ML models.

In [ ]:
@dataclass
class ModelConfig:
    """Transformer model configuration"""
    d_model: int = 64          # Embedding dimension
    n_heads: int = 2           # Number of attention heads
    n_layers: int = 3          # Number of transformer layers
    dropout: float = 0.1       # Dropout rate
    learning_rate: float = 0.001
    epochs: int = 100
    batch_size: int = 32

@dataclass
class LDAConfig:
    """LDA topic modeling configuration"""
    n_topics: int = 5          # Number of topics to discover
    n_iterations: int = 100    # Gibbs sampling iterations
    alpha: float = 0.1         # Document-topic prior
    beta: float = 0.01         # Topic-word prior

# Five-tier risk classification thresholds
RISK_THRESHOLDS = {
    'low': (0.0, 0.20),
    'early_warning': (0.20, 0.40),
    'moderate': (0.40, 0.60),
    'high': (0.60, 0.80),
    'critical': (0.80, 1.0)
}

RISK_COLORS = {
    'low': '#22c55e',
    'early_warning': '#3b82f6',
    'moderate': '#f59e0b',
    'high': '#f97316',
    'critical': '#ef4444'
}

print("✅ Configuration loaded!")
print(f"\nTransformer Config:")
config = ModelConfig()
print(f"  d_model={config.d_model}, n_heads={config.n_heads}, n_layers={config.n_layers}")
print(f"  epochs={config.epochs}, lr={config.learning_rate}")
print(f"\nLDA Config:")
lda_config = LDAConfig()
print(f"  n_topics={lda_config.n_topics}, iterations={lda_config.n_iterations}")
print(f"  alpha={lda_config.alpha}, beta={lda_config.beta}")
print(f"\nRisk Thresholds:")
for level, (low, high) in RISK_THRESHOLDS.items():
    print(f"  {level.upper():15s}: {low:.0%} - {high:.0%}")

---

## 🏗️ Step 2: Transformer Encoder Architecture

The Transformer Encoder uses **multi-head self-attention** to learn relationships between features.

### Key Concepts:
- **Self-Attention**: Each feature "attends" to all other features, learning which combinations matter
- **Multi-Head**: Multiple parallel attention operations capture different types of relationships
- **Residual Connections**: Skip connections that prevent vanishing gradients
- **Layer Normalization**: Stabilizes training by normalizing activations

### Architecture:
```
Input → Embedding → [Attention + FFN] × N_layers → Sigmoid → Probability
```

### Mathematical Foundation:
$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

In [ ]:
class TransformerEncoder:
    """
    Transformer Encoder for Tabular Data Prediction
    
    Architecture:
    - Multi-head self-attention mechanism
    - Position-wise feed-forward networks
    - Layer normalization and residual connections
    - SELU activation function
    
    Reference: "Attention Is All You Need" (Vaswani et al., 2017)
    """
    
    def __init__(self, config: ModelConfig = ModelConfig()):
        self.config = config
        self.weights = {}
        self.is_trained = False
        
    def _initialize_weights(self, input_dim: int):
        """Initialize transformer weights using Xavier initialization"""
        d_model = self.config.d_model
        
        # Input embedding
        self.weights['W_embed'] = np.random.randn(input_dim, d_model) * np.sqrt(2.0 / input_dim)
        self.weights['b_embed'] = np.zeros(d_model)
        
        # Multi-head attention weights for each layer
        for layer in range(self.config.n_layers):
            self.weights[f'W_q_{layer}'] = np.random.randn(d_model, d_model) * np.sqrt(2.0 / d_model)
            self.weights[f'W_k_{layer}'] = np.random.randn(d_model, d_model) * np.sqrt(2.0 / d_model)
            self.weights[f'W_v_{layer}'] = np.random.randn(d_model, d_model) * np.sqrt(2.0 / d_model)
            self.weights[f'W_o_{layer}'] = np.random.randn(d_model, d_model) * np.sqrt(2.0 / d_model)
            self.weights[f'W_ff1_{layer}'] = np.random.randn(d_model, d_model * 4) * np.sqrt(2.0 / d_model)
            self.weights[f'b_ff1_{layer}'] = np.zeros(d_model * 4)
            self.weights[f'W_ff2_{layer}'] = np.random.randn(d_model * 4, d_model) * np.sqrt(2.0 / (d_model * 4))
            self.weights[f'b_ff2_{layer}'] = np.zeros(d_model)
            self.weights[f'gamma_attn_{layer}'] = np.ones(d_model)
            self.weights[f'beta_attn_{layer}'] = np.zeros(d_model)
            self.weights[f'gamma_ff_{layer}'] = np.ones(d_model)
            self.weights[f'beta_ff_{layer}'] = np.zeros(d_model)
        
        self.weights['W_out'] = np.random.randn(d_model, 1) * np.sqrt(2.0 / d_model)
        self.weights['b_out'] = np.zeros(1)
        
    def _selu(self, x: np.ndarray) -> np.ndarray:
        """SELU activation: Self-Normalizing Exponential Linear Unit"""
        alpha = 1.6732632423543772848170429916717
        scale = 1.0507009873554804934193349852946
        return scale * np.where(x > 0, x, alpha * (np.exp(x) - 1))
    
    def _layer_norm(self, x, gamma, beta, eps=1e-6):
        """Layer Normalization: normalizes across features"""
        mean = np.mean(x, axis=-1, keepdims=True)
        std = np.std(x, axis=-1, keepdims=True)
        return gamma * (x - mean) / (std + eps) + beta
    
    def _scaled_dot_product_attention(self, Q, K, V):
        """Attention(Q, K, V) = softmax(QK^T / sqrt(d_k)) * V"""
        d_k = Q.shape[-1]
        scores = np.dot(Q, K.T) / np.sqrt(d_k)
        exp_scores = np.exp(scores - np.max(scores, axis=-1, keepdims=True))
        attention_weights = exp_scores / np.sum(exp_scores, axis=-1, keepdims=True)
        return np.dot(attention_weights, V), attention_weights
    
    def _multi_head_attention(self, x, layer):
        """Multi-head self-attention mechanism"""
        Q = np.dot(x, self.weights[f'W_q_{layer}'])
        K = np.dot(x, self.weights[f'W_k_{layer}'])
        V = np.dot(x, self.weights[f'W_v_{layer}'])
        attention_output, attention_weights = self._scaled_dot_product_attention(Q, K, V)
        output = np.dot(attention_output, self.weights[f'W_o_{layer}'])
        return output, attention_weights
    
    def _feed_forward(self, x, layer):
        """Position-wise Feed-Forward Network"""
        hidden = np.dot(x, self.weights[f'W_ff1_{layer}']) + self.weights[f'b_ff1_{layer}']
        hidden = self._selu(hidden)
        output = np.dot(hidden, self.weights[f'W_ff2_{layer}']) + self.weights[f'b_ff2_{layer}']
        return output
    
    def forward(self, x):
        """Forward pass through the complete transformer encoder"""
        hidden = np.dot(x, self.weights['W_embed']) + self.weights['b_embed']
        attention_maps = {}
        
        for layer in range(self.config.n_layers):
            # Self-attention with residual
            attn_output, attn_weights = self._multi_head_attention(hidden, layer)
            hidden = hidden + attn_output
            hidden = self._layer_norm(hidden, self.weights[f'gamma_attn_{layer}'], self.weights[f'beta_attn_{layer}'])
            attention_maps[f'layer_{layer}'] = attn_weights
            
            # Feed-forward with residual
            ff_output = self._feed_forward(hidden, layer)
            hidden = hidden + ff_output
            hidden = self._layer_norm(hidden, self.weights[f'gamma_ff_{layer}'], self.weights[f'beta_ff_{layer}'])
        
        logits = np.dot(hidden, self.weights['W_out']) + self.weights['b_out']
        probabilities = 1 / (1 + np.exp(-logits))  # Sigmoid
        return probabilities.flatten(), attention_maps
    
    def fit(self, X, y, verbose=True):
        """Train the transformer model"""
        self._initialize_weights(X.shape[1])
        n_samples = X.shape[0]
        best_loss = float('inf')
        history = {'loss': [], 'accuracy': []}
        
        if verbose:
            print("\n" + "="*60)
            print("🏋️ TRANSFORMER ENCODER TRAINING")
            print("="*60)
            print(f"  d_model={self.config.d_model}, heads={self.config.n_heads}, layers={self.config.n_layers}")
            print(f"  Training samples: {n_samples}")
            print("-"*60)
        
        for epoch in range(self.config.epochs):
            predictions, _ = self.forward(X)
            epsilon = 1e-7
            loss = -np.mean(y * np.log(predictions + epsilon) + (1 - y) * np.log(1 - predictions + epsilon))
            accuracy = np.mean((predictions > 0.5) == y) * 100
            
            gradient = (predictions - y).reshape(-1, 1)
            self.weights['W_out'] -= self.config.learning_rate * np.dot(
                np.dot(X, self.weights['W_embed']).T, gradient
            ) / n_samples
            
            history['loss'].append(loss)
            history['accuracy'].append(accuracy)
            
            if loss < best_loss:
                best_loss = loss
            
            if verbose and (epoch + 1) % 20 == 0:
                print(f"  Epoch {epoch+1:3d}/{self.config.epochs} | Loss: {loss:.4f} | Accuracy: {accuracy:.1f}%")
        
        self.is_trained = True
        if verbose:
            print("-"*60)
            print(f"  ✅ Training complete. Best loss: {best_loss:.4f}")
            print("="*60)
        
        return history
    
    def predict(self, X):
        """Predict attrition probabilities"""
        if not self.is_trained:
            raise ValueError("Model must be trained before prediction")
        return self.forward(X)

print("✅ TransformerEncoder class defined!")

---

## 🔍 Step 3: SHAP Explainability Engine

**SHAP (SHapley Additive exPlanations)** explains each prediction by assigning a contribution score to every feature.

### Key Concept:
Think of features as "players" in a cooperative game. The prediction is the "payout." SHAP fairly distributes the payout based on each player's marginal contribution.

### Formula:
$$f(x) = \phi_0 + \sum_{i=1}^{M} \phi_i(x)$$

Where $\phi_0$ is the base value and $\phi_i$ is the Shapley value of feature $i$.

### Shapley Value Calculation:
$$\phi_i(x) = \sum_{S \subseteq N \setminus \{i\}} \frac{|S|!(|N|-|S|-1)!}{|N|!} [f(S \cup \{i\}) - f(S)]$$

In [ ]:
class SHAPExplainer:
    """
    SHAP Explainability Engine
    Calculates Shapley values for feature attribution.
    Reference: Lundberg & Lee (2017)
    """
    
    def __init__(self, model: TransformerEncoder, feature_names: List[str]):
        self.model = model
        self.feature_names = feature_names
        self.baseline = None
        
    def _get_baseline_prediction(self, X):
        predictions, _ = self.model.predict(X)
        return np.mean(predictions)
    
    def _calculate_shapley_value(self, x, feature_idx, X_background, n_samples=100):
        """Calculate Shapley value using Monte Carlo sampling"""
        n_features = len(x)
        shapley_values = []
        
        for _ in range(n_samples):
            perm = np.random.permutation(n_features)
            feature_pos = np.where(perm == feature_idx)[0][0]
            bg_idx = np.random.randint(len(X_background))
            
            x_with = X_background[bg_idx].copy()
            x_without = X_background[bg_idx].copy()
            
            for j in range(feature_pos + 1):
                x_with[perm[j]] = x[perm[j]]
            for j in range(feature_pos):
                x_without[perm[j]] = x[perm[j]]
            
            pred_with, _ = self.model.predict(x_with.reshape(1, -1))
            pred_without, _ = self.model.predict(x_without.reshape(1, -1))
            shapley_values.append(pred_with[0] - pred_without[0])
        
        return np.mean(shapley_values)
    
    def explain(self, X, X_background=None, max_samples=50):
        """Generate SHAP explanations"""
        if X_background is None:
            X_background = X
            
        self.baseline = self._get_baseline_prediction(X_background)
        
        # Limit samples for Colab performance
        n_explain = min(X.shape[0], max_samples)
        n_features = X.shape[1]
        shap_values = np.zeros((n_explain, n_features))
        
        print("\n" + "="*60)
        print("🔍 SHAP EXPLAINABILITY ANALYSIS")
        print("="*60)
        print(f"  Calculating Shapley values for {n_explain} samples...")
        print(f"  Features: {n_features}")
        print("-"*60)
        
        for i in range(n_explain):
            if (i + 1) % 10 == 0:
                print(f"  Processing sample {i+1}/{n_explain}...")
            for j in range(n_features):
                shap_values[i, j] = self._calculate_shapley_value(X[i], j, X_background, n_samples=50)
        
        # Global feature importance
        feature_importance = np.mean(np.abs(shap_values), axis=0)
        feature_importance_normalized = feature_importance / (np.sum(feature_importance) + 1e-10)
        
        importance_ranking = []
        for idx in np.argsort(feature_importance)[::-1]:
            importance_ranking.append({
                'feature': self.feature_names[idx],
                'importance': float(feature_importance_normalized[idx]),
                'mean_shap': float(np.mean(shap_values[:, idx])),
                'direction': 'increases risk' if np.mean(shap_values[:, idx]) > 0 else 'decreases risk'
            })
        
        print("-"*60)
        print("📊 Top Feature Importance (SHAP):")
        for i, feat in enumerate(importance_ranking[:10]):
            direction = "↑ risk" if feat['direction'] == 'increases risk' else "↓ risk"
            bar = "█" * int(feat['importance'] * 50)
            print(f"  {i+1:2d}. {feat['feature']:25s} {bar} {feat['importance']:.1%} ({direction})")
        print("="*60)
        
        return {
            'shap_values': shap_values,
            'feature_importance': importance_ranking,
            'base_value': self.baseline,
            'feature_names': self.feature_names
        }

print("✅ SHAPExplainer class defined!")

---

## 📝 Step 4: LDA Topic Modeling

**LDA (Latent Dirichlet Allocation)** discovers hidden themes in employee reviews.

### How it works:
1. Each **document** (review) is a mixture of **topics**
2. Each **topic** is a distribution over **words**
3. **Gibbs Sampling** iteratively assigns words to topics

### Sampling Formula:
$$p(z_i = k | \mathbf{z}_{-i}, \mathbf{w}) \propto (n_{d,k} + \alpha) \times \frac{n_{k,w} + \beta}{n_k + V\beta}$$

Where:
- $n_{d,k}$ = words in document $d$ assigned to topic $k$
- $n_{k,w}$ = times word $w$ assigned to topic $k$
- $\alpha$ = document-topic prior
- $\beta$ = topic-word prior

In [ ]:
class LDATopicModel:
    """
    Latent Dirichlet Allocation using Collapsed Gibbs Sampling
    Reference: Blei, Ng, Jordan (2003)
    """
    
    def __init__(self, config: LDAConfig = LDAConfig()):
        self.config = config
        self.vocabulary = {}
        self.word_topic_counts = None
        self.doc_topic_counts = None
        self.topic_counts = None
        self.topic_word_dist = None
        
    def _tokenize(self, text: str) -> List[str]:
        """Tokenize and preprocess text"""
        import re
        text = text.lower()
        words = re.findall(r'\b[a-z]{3,}\b', text)
        stop_words = {
            'the', 'and', 'for', 'are', 'but', 'not', 'you', 'all', 'can',
            'had', 'her', 'was', 'one', 'our', 'out', 'has', 'have', 'been',
            'would', 'could', 'should', 'will', 'just', 'than', 'them', 'this',
            'that', 'with', 'they', 'from', 'were', 'which', 'there', 'their',
            'what', 'about', 'when', 'make', 'like', 'very', 'some', 'also',
            'company', 'work', 'employee', 'employees', 'working', 'place'
        }
        return [w for w in words if w not in stop_words]
    
    def _build_vocabulary(self, documents):
        word_counts = Counter()
        for doc in documents:
            word_counts.update(self._tokenize(doc))
        vocab = {word: idx for idx, (word, count) in enumerate(word_counts.most_common()) if count >= 2}
        doc_words = []
        for doc in documents:
            indices = [vocab[w] for w in self._tokenize(doc) if w in vocab]
            doc_words.append(indices)
        return vocab, doc_words
    
    def _analyze_sentiment(self, text: str) -> float:
        positive = {'good', 'great', 'excellent', 'amazing', 'wonderful', 'best', 'happy', 'growth', 'opportunity', 'learning', 'support'}
        negative = {'bad', 'poor', 'terrible', 'awful', 'worst', 'hate', 'stress', 'toxic', 'politics', 'underpaid', 'burnout', 'quit'}
        tokens = set(self._tokenize(text))
        pos = len(tokens & positive)
        neg = len(tokens & negative)
        return (pos - neg) / max(pos + neg, 1)
    
    def fit_transform(self, documents: List[str]) -> Dict:
        """Run LDA topic modeling"""
        print("\n" + "="*60)
        print("📝 LDA TOPIC MODELING")
        print("="*60)
        print(f"  Documents: {len(documents)}")
        print(f"  Topics: {self.config.n_topics}")
        print(f"  Iterations: {self.config.n_iterations}")
        print("-"*60)
        
        self.vocabulary, doc_words = self._build_vocabulary(documents)
        n_vocab = len(self.vocabulary)
        print(f"  Vocabulary size: {n_vocab}")
        
        if n_vocab == 0:
            return {'topics': [], 'doc_topics': np.array([])}
        
        # Initialize counts
        n_topics = self.config.n_topics
        self.word_topic_counts = np.zeros((n_vocab, n_topics)) + self.config.beta
        self.doc_topic_counts = np.zeros((len(doc_words), n_topics)) + self.config.alpha
        self.topic_counts = np.zeros(n_topics) + n_vocab * self.config.beta
        
        # Random initialization
        topic_assignments = []
        for d, words in enumerate(doc_words):
            doc_topics = []
            for w in words:
                topic = np.random.randint(n_topics)
                doc_topics.append(topic)
                self.word_topic_counts[w, topic] += 1
                self.doc_topic_counts[d, topic] += 1
                self.topic_counts[topic] += 1
            topic_assignments.append(doc_topics)
        
        # Gibbs Sampling
        print("  Running Gibbs sampling...")
        for iteration in range(self.config.n_iterations):
            if (iteration + 1) % 20 == 0:
                print(f"    Iteration {iteration+1}/{self.config.n_iterations}")
            for d, words in enumerate(doc_words):
                for i, w in enumerate(words):
                    old_topic = topic_assignments[d][i]
                    self.word_topic_counts[w, old_topic] -= 1
                    self.doc_topic_counts[d, old_topic] -= 1
                    self.topic_counts[old_topic] -= 1
                    
                    topic_probs = self.doc_topic_counts[d] * self.word_topic_counts[w] / self.topic_counts
                    topic_probs /= topic_probs.sum()
                    
                    new_topic = np.random.choice(n_topics, p=topic_probs)
                    topic_assignments[d][i] = new_topic
                    self.word_topic_counts[w, new_topic] += 1
                    self.doc_topic_counts[d, new_topic] += 1
                    self.topic_counts[new_topic] += 1
        
        self.topic_word_dist = self.word_topic_counts.T / self.word_topic_counts.sum(axis=0)
        
        # Extract topics
        inv_vocab = {idx: word for word, idx in self.vocabulary.items()}
        topics = []
        
        print("\n📋 Extracted Topics:")
        print("-"*60)
        for k in range(n_topics):
            word_indices = np.argsort(self.topic_word_dist[k])[::-1][:10]
            keywords = [(inv_vocab[idx], float(self.topic_word_dist[k, idx])) for idx in word_indices]
            
            # Calculate sentiment
            topic_docs = [documents[d] for d in range(len(documents)) if np.argmax(self.doc_topic_counts[d]) == k]
            avg_sentiment = np.mean([self._analyze_sentiment(doc) for doc in topic_docs]) if topic_docs else 0
            
            topics.append({
                'id': k,
                'keywords': keywords,
                'sentiment': avg_sentiment,
                'label': 'Positive' if avg_sentiment > 0.1 else 'Negative' if avg_sentiment < -0.1 else 'Neutral',
                'doc_count': len(topic_docs)
            })
            
            kw_str = ", ".join([f"{w} ({s:.3f})" for w, s in keywords[:5]])
            emoji = "😊" if avg_sentiment > 0.1 else "😟" if avg_sentiment < -0.1 else "😐"
            print(f"  Topic {k+1}: {kw_str}")
            print(f"           Sentiment: {avg_sentiment:.3f} {emoji} ({topics[-1]['label']})")
        
        print("="*60)
        return {'topics': topics, 'doc_topics': self.doc_topic_counts / self.doc_topic_counts.sum(axis=1, keepdims=True)}

print("✅ LDATopicModel class defined!")

---

## 🎯 Step 5: Risk Classification & Recommendations

Converts raw probabilities into five actionable risk tiers with specific intervention strategies.

In [ ]:
def classify_risk(probability: float) -> Dict:
    """Five-Tier Risk Classification"""
    actions = {
        'low': 'Continue monitoring. Maintain current engagement.',
        'early_warning': 'Schedule check-in. Review recent feedback.',
        'moderate': 'Initiate retention discussion. Consider role adjustments.',
        'high': 'Urgent intervention. Discuss career development.',
        'critical': 'Immediate action. Executive-level engagement.'
    }
    for level, (low, high) in RISK_THRESHOLDS.items():
        if low <= probability < high:
            return {'level': level, 'probability': probability, 'range': f"{low:.0%}-{high:.0%}", 'action': actions[level]}
    return {'level': 'critical', 'probability': probability, 'range': '80%-100%', 'action': actions['critical']}

print("✅ Risk classification defined!")
print("\nExample classifications:")
for p in [0.12, 0.35, 0.52, 0.71, 0.88]:
    r = classify_risk(p)
    print(f"  p={p:.2f} → {r['level'].upper():15s} | {r['action']}")

---

## 📊 Step 6: Data Preprocessing

The preprocessor handles multiple dataset formats (IBM HR, Kaggle, AmbitionBox) using column mapping.

In [ ]:
class DataPreprocessor:
    """Preprocess HR data for model training — supports IBM, Kaggle, and AmbitionBox formats"""
    
    COLUMN_MAPPINGS = {
        'satisfaction': ['JobSatisfaction', 'satisfaction_level', 'Rating', 'EnvironmentSatisfaction', 'Overall_rating', 'overall_rating'],
        'tenure': ['YearsAtCompany', 'tenure', 'TotalWorkingYears', 'YearsInCurrentRole', 'time_spend_company'],
        'overtime': ['OverTime', 'average_montly_hours', 'WorkLifeBalance', 'work_life_balance'],
        'salary': ['MonthlyIncome', 'salary', 'DailyRate', 'HourlyRate', 'salary_and_benefits'],
        'department': ['Department', 'department', 'JobRole'],
        'promotion': ['YearsSinceLastPromotion', 'promotion_last_5years', 'career_growth'],
        'projects': ['NumCompaniesWorked', 'number_project', 'skill_development'],
        'attrition': ['Attrition', 'left', 'Status']
    }
    
    def __init__(self):
        self.feature_names = []
        
    def _find_column(self, df, candidates):
        for col in candidates:
            for df_col in df.columns:
                if df_col.lower() == col.lower():
                    return df_col
        return None
    
    def fit_transform(self, df):
        """Preprocess dataframe and extract features"""
        print("\n" + "="*60)
        print("📊 DATA PREPROCESSING")
        print("="*60)
        print(f"  Original shape: {df.shape}")
        print(f"  Columns: {list(df.columns[:10])}{'...' if len(df.columns) > 10 else ''}")
        print("-"*60)
        
        features = []
        feature_names = []
        
        for feature_type, candidates in self.COLUMN_MAPPINGS.items():
            if feature_type == 'attrition':
                continue
            col = self._find_column(df, candidates)
            if col is not None:
                values = df[col].copy()
                if values.dtype == 'object':
                    if feature_type == 'overtime':
                        values = values.map({'Yes': 1, 'No': 0, True: 1, False: 0}).fillna(0)
                    elif feature_type == 'department':
                        for dept in values.unique():
                            if pd.notna(dept):
                                features.append((values == dept).astype(float).values)
                                feature_names.append(f'dept_{dept}')
                        continue
                    else:
                        mapping = {v: i for i, v in enumerate(values.unique())}
                        values = values.map(mapping).fillna(0)
                
                values = pd.to_numeric(values, errors='coerce').fillna(0)
                if values.std() > 0:
                    values = (values - values.mean()) / values.std()
                features.append(values.values)
                feature_names.append(feature_type)
                print(f"  ✓ Added feature: {feature_type} (from column: {col})")
        
        # Extract target
        target_col = self._find_column(df, self.COLUMN_MAPPINGS['attrition'])
        if target_col is not None:
            y = df[target_col].copy()
            if y.dtype == 'object':
                y = y.map({'Yes': 1, 'No': 0, 'Left': 1, 'Stayed': 0}).fillna(0)
            y = y.values.astype(float)
            print(f"  ✓ Target: {target_col} (attrition rate: {y.mean():.1%})")
        else:
            print("  ⚠ No attrition column found — generating synthetic target")
            y = np.random.binomial(1, 0.15, len(df))
        
        X = np.column_stack(features) if features else np.random.randn(len(df), 5)
        self.feature_names = feature_names if feature_names else [f'feature_{i}' for i in range(X.shape[1])]
        
        print(f"\n  Processed shape: X={X.shape}, y={y.shape}")
        print(f"  Features: {self.feature_names}")
        print("="*60)
        return X, y, self.feature_names

print("✅ DataPreprocessor class defined!")

---

## 🏭 Step 7: Generate Sample Data

Generate a realistic HR dataset for demonstration. You can also upload your own CSV below.

In [ ]:
def generate_sample_data(n_samples=200):
    """Generate sample HR dataset for demonstration"""
    np.random.seed(42)
    departments = ['Sales', 'Engineering', 'HR', 'Marketing', 'Finance']
    
    data = {
        'EmployeeID': range(1, n_samples + 1),
        'Department': np.random.choice(departments, n_samples),
        'JobSatisfaction': np.random.randint(1, 5, n_samples),
        'YearsAtCompany': np.random.randint(0, 20, n_samples),
        'MonthlyIncome': np.random.randint(3000, 15000, n_samples),
        'OverTime': np.random.choice(['Yes', 'No'], n_samples, p=[0.3, 0.7]),
        'WorkLifeBalance': np.random.randint(1, 5, n_samples),
        'YearsSinceLastPromotion': np.random.randint(0, 10, n_samples),
        'EnvironmentSatisfaction': np.random.randint(1, 5, n_samples),
    }
    
    attrition_prob = (
        (5 - data['JobSatisfaction']) * 0.1 +
        (data['OverTime'] == 'Yes').astype(int) * 0.2 +
        (data['YearsAtCompany'] < 2).astype(int) * 0.15 +
        (data['WorkLifeBalance'] < 2).astype(int) * 0.1 +
        np.random.uniform(0, 0.1, n_samples)
    )
    data['Attrition'] = ['Yes' if np.random.random() < p else 'No' for p in np.clip(attrition_prob, 0, 1)]
    
    issues = ['overtime is excessive', 'promotion opportunities are rare', 'workload is high',
              'communication could improve', 'benefits need updating', 'stress levels are concerning']
    templates = ['Good work environment but {}', 'I enjoy my work, however {}', 'Career growth is limited and {}']
    data['Review'] = [np.random.choice(templates).format(np.random.choice(issues)) for _ in range(n_samples)]
    
    return pd.DataFrame(data)

# Generate and display sample data
sample_df = generate_sample_data(200)
print(f"📋 Sample dataset generated: {sample_df.shape[0]} employees, {sample_df.shape[1]} columns")
print(f"   Attrition rate: {(sample_df['Attrition'] == 'Yes').mean():.1%}")
print(f"   Departments: {sample_df['Department'].unique().tolist()}")
sample_df.head(10)

---

## 📤 Step 8: Upload Your Own Dataset (Optional)

Upload a CSV file to analyze your own HR data. Supported formats:
- **IBM HR Analytics** (columns: Attrition, JobSatisfaction, Department, etc.)
- **Kaggle HR Analytics** (columns: left, satisfaction_level, number_project, etc.)
- **AmbitionBox Reviews** (columns: Overall_rating, work_life_balance, Likes, Dislikes, etc.)

**If you skip this cell**, the analysis will use the sample data generated above.

In [ ]:
# === OPTION 1: Upload a CSV file ===
# Uncomment the following lines to upload your own data:

# from google.colab import files
# uploaded = files.upload()
# filename = list(uploaded.keys())[0]
# df = pd.read_csv(filename)
# print(f"\n✅ Loaded: {filename}")
# print(f"   Shape: {df.shape}")
# print(f"   Columns: {list(df.columns)}")
# df.head()

# === OPTION 2: Use sample data (default) ===
df = sample_df
print("Using sample data (200 employees)")
print("💡 To use your own data, uncomment the upload section above")

---

## 🚀 Step 9: Run the Complete Analysis Pipeline

This executes the full pipeline:
1. Data Preprocessing
2. Transformer Training
3. Prediction
4. SHAP Analysis
5. LDA Topic Modeling
6. Risk Classification
7. Recommendations

In [ ]:
print("\n" + "="*70)
print("   🧠 EMPLOYEE ATTRITION PREDICTION SYSTEM")
print("   XAI-Powered Models for Managerial Decision-Making")
print("="*70)

# Step 1: Preprocess
preprocessor = DataPreprocessor()
X, y, feature_names = preprocessor.fit_transform(df)

# Step 2: Train Transformer
config = ModelConfig(epochs=100, n_layers=3)
model = TransformerEncoder(config)
history = model.fit(X, y)

# Step 3: Predict
predictions, attention_maps = model.predict(X)

# Step 4: SHAP
explainer = SHAPExplainer(model, feature_names)
shap_results = explainer.explain(X, max_samples=min(50, len(X)))

# Step 5: LDA (if text reviews available)
lda_results = None
text_col = None
for col_name in ['Review', 'review', 'Likes', 'likes', 'Dislikes', 'dislikes']:
    if col_name in df.columns:
        text_col = col_name
        break

if text_col:
    documents = df[text_col].dropna().tolist()
    if len(documents) > 10:
        lda_model = LDATopicModel(LDAConfig(n_topics=5, n_iterations=50))
        lda_results = lda_model.fit_transform(documents)

# Step 6: Risk Classification
risk_classifications = [classify_risk(p) for p in predictions]
risk_counts = Counter(r['level'] for r in risk_classifications)

# Summary
print("\n" + "="*70)
print("   📊 ANALYSIS SUMMARY")
print("="*70)
print(f"  Total Employees: {len(df)}")
print(f"  Historical Attrition Rate: {y.mean():.1%}")
print(f"  Model Accuracy: {np.mean((predictions > 0.5) == y) * 100:.1f}%")
high_risk = risk_counts.get('high', 0) + risk_counts.get('critical', 0)
print(f"  High/Critical Risk: {high_risk} employees")
print(f"\n  Risk Distribution:")
for level in ['low', 'early_warning', 'moderate', 'high', 'critical']:
    count = risk_counts.get(level, 0)
    pct = count / len(predictions) * 100
    bar = "█" * int(pct / 2) + "░" * (50 - int(pct / 2))
    print(f"    {level.upper():15s}: {bar} {count:4d} ({pct:.1f}%)")
print("="*70)

---

## 📈 Step 10: Visualizations

Professional charts showing the analysis results.

In [ ]:
# === 1. Training History ===
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(history['loss'], color='#ef4444', linewidth=2)
axes[0].set_title('Training Loss (Binary Cross-Entropy)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].grid(True, alpha=0.3)

axes[1].plot(history['accuracy'], color='#22c55e', linewidth=2)
axes[1].set_title('Training Accuracy', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
print("📈 Training curves show model convergence over epochs")

In [ ]:
# === 2. Risk Distribution (Donut Chart) ===
fig, ax = plt.subplots(figsize=(8, 8))

labels = []
sizes = []
colors = []
explode_vals = []

for level in ['low', 'early_warning', 'moderate', 'high', 'critical']:
    count = risk_counts.get(level, 0)
    if count > 0:
        labels.append(f"{level.replace('_', ' ').title()}\n({count})")
        sizes.append(count)
        colors.append(RISK_COLORS[level])
        explode_vals.append(0.05 if level in ['high', 'critical'] else 0)

wedges, texts, autotexts = ax.pie(
    sizes, labels=labels, colors=colors, explode=explode_vals,
    autopct='%1.1f%%', pctdistance=0.75,
    wedgeprops={'width': 0.5, 'edgecolor': 'white', 'linewidth': 2},
    textprops={'fontsize': 11}
)
for autotext in autotexts:
    autotext.set_fontweight('bold')

ax.set_title('Five-Tier Risk Distribution', fontsize=16, fontweight='bold', pad=20)
centre = plt.Circle((0, 0), 0.3, fc='white')
ax.add_patch(centre)
ax.text(0, 0, f'{len(predictions)}\nTotal', ha='center', va='center', fontsize=16, fontweight='bold')

plt.tight_layout()
plt.show()
print("🍩 Donut chart shows the distribution across all five risk tiers")

In [ ]:
# === 3. Feature Importance (SHAP) ===
fig, ax = plt.subplots(figsize=(12, 6))

top_features = shap_results['feature_importance'][:10]
feat_names = [f['feature'] for f in top_features][::-1]
feat_values = [f['importance'] for f in top_features][::-1]
feat_colors = ['#ef4444' if f['direction'] == 'increases risk' else '#22c55e' for f in top_features][::-1]

bars = ax.barh(feat_names, feat_values, color=feat_colors, edgecolor='white', linewidth=1, height=0.6)
ax.set_title('Global Feature Importance (SHAP Values)', fontsize=16, fontweight='bold')
ax.set_xlabel('Mean |SHAP Value| (Normalized Importance)', fontsize=12)

for bar, val in zip(bars, feat_values):
    ax.text(bar.get_width() + 0.005, bar.get_y() + bar.get_height()/2,
            f'{val:.1%}', va='center', fontsize=11, fontweight='bold')

# Legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#ef4444', label='Increases Attrition Risk'),
    Patch(facecolor='#22c55e', label='Decreases Attrition Risk')
]
ax.legend(handles=legend_elements, loc='lower right', fontsize=11)
ax.grid(True, axis='x', alpha=0.3)

plt.tight_layout()
plt.show()
print("📊 SHAP bar chart shows which features drive attrition predictions")
print("   🔴 Red = increases risk | 🟢 Green = decreases risk")

In [ ]:
# === 4. Risk Score Distribution (Histogram with zones) ===
fig, ax = plt.subplots(figsize=(12, 6))

# Background risk zones
zone_colors = ['#dcfce7', '#dbeafe', '#fef3c7', '#fed7aa', '#fecaca']
zone_labels = ['Low', 'Early\nWarning', 'Moderate', 'High', 'Critical']
boundaries = [0, 0.2, 0.4, 0.6, 0.8, 1.0]

for i in range(5):
    ax.axvspan(boundaries[i], boundaries[i+1], alpha=0.4, color=zone_colors[i])
    ax.text((boundaries[i] + boundaries[i+1]) / 2, ax.get_ylim()[1] * 0.02,
            zone_labels[i], ha='center', fontsize=9, color='gray', style='italic')

ax.hist(predictions, bins=30, color='#3b82f6', edgecolor='white', alpha=0.8, linewidth=1)
ax.set_title('Attrition Risk Score Distribution', fontsize=16, fontweight='bold')
ax.set_xlabel('Risk Score (Probability)', fontsize=12)
ax.set_ylabel('Number of Employees', fontsize=12)
ax.set_xlim(0, 1)

# Re-add zone labels after histogram sets y-limits
ylim = ax.get_ylim()
for i in range(5):
    ax.text((boundaries[i] + boundaries[i+1]) / 2, ylim[1] * 0.95,
            zone_labels[i], ha='center', fontsize=9, color='gray', fontweight='bold')

plt.tight_layout()
plt.show()
print("📊 Histogram shows how risk scores are distributed across the workforce")

In [ ]:
# === 5. Individual Employee Explanation (SHAP Waterfall) ===
# Show SHAP explanation for the highest-risk employee
highest_risk_idx = np.argmax(predictions)
employee_shap = shap_results['shap_values'][min(highest_risk_idx, len(shap_results['shap_values'])-1)]

fig, ax = plt.subplots(figsize=(10, 6))

sorted_idx = np.argsort(np.abs(employee_shap))[::-1][:8]
feat_labels = [feature_names[i] if i < len(feature_names) else f'Feature {i}' for i in sorted_idx]
shap_vals = [employee_shap[i] for i in sorted_idx]
bar_colors = ['#ef4444' if v > 0 else '#22c55e' for v in shap_vals]

bars = ax.barh(feat_labels[::-1], [v for v in shap_vals[::-1]], 
               color=bar_colors[::-1], edgecolor='white', height=0.6)

ax.set_title(f'SHAP Explanation — Employee #{highest_risk_idx+1} (Risk: {predictions[highest_risk_idx]:.1%})',
             fontsize=14, fontweight='bold')
ax.set_xlabel('SHAP Value (Impact on Risk)', fontsize=12)
ax.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
ax.grid(True, axis='x', alpha=0.3)

plt.tight_layout()
plt.show()
print(f"🔍 Individual explanation for the highest-risk employee")
print(f"   Risk: {risk_classifications[highest_risk_idx]['level'].upper()} ({predictions[highest_risk_idx]:.1%})")
print(f"   Action: {risk_classifications[highest_risk_idx]['action']}")

In [ ]:
# === 6. Department Analysis ===
if 'Department' in df.columns:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    dept_risks = {}
    for dept in df['Department'].unique():
        dept_mask = (df['Department'] == dept).values
        dept_preds = predictions[dept_mask]
        dept_risks[dept] = {
            'total': len(dept_preds),
            'at_risk': sum(1 for p in dept_preds if p > 0.5),
            'avg_risk': np.mean(dept_preds)
        }
    
    depts = sorted(dept_risks.keys(), key=lambda d: dept_risks[d]['avg_risk'], reverse=True)
    x = np.arange(len(depts))
    width = 0.35
    
    ax.bar(x - width/2, [dept_risks[d]['total'] for d in depts], width, 
           label='Total', color='#3b82f6', alpha=0.7)
    ax.bar(x + width/2, [dept_risks[d]['at_risk'] for d in depts], width,
           label='At Risk (>50%)', color='#ef4444', alpha=0.7)
    
    ax.set_title('Department-wise Attrition Risk', fontsize=16, fontweight='bold')
    ax.set_ylabel('Number of Employees', fontsize=12)
    ax.set_xticks(x)
    ax.set_xticklabels(depts, fontsize=11)
    ax.legend(fontsize=11)
    ax.grid(True, axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    print("📊 Department analysis shows which teams have highest attrition risk")

---

## 💾 Step 11: Save Results

Export the complete analysis results to a JSON file.

In [ ]:
# Compile results
results = {
    'summary': {
        'total_employees': len(df),
        'attrition_rate': float(y.mean()),
        'model_accuracy': float(np.mean((predictions > 0.5) == y)),
        'high_risk_count': int(risk_counts.get('high', 0) + risk_counts.get('critical', 0)),
    },
    'risk_distribution': {level: int(risk_counts.get(level, 0)) for level in RISK_THRESHOLDS},
    'feature_importance': shap_results['feature_importance'][:10],
    'model_config': {
        'd_model': config.d_model,
        'n_heads': config.n_heads,
        'n_layers': config.n_layers,
        'epochs': config.epochs,
    }
}

# Save to file
with open('attrition_results.json', 'w') as f:
    json.dump(results, f, indent=2, default=str)

print("✅ Results saved to 'attrition_results.json'")
print("\n📋 Results Summary:")
print(json.dumps(results['summary'], indent=2))
print("\n📊 Risk Distribution:")
print(json.dumps(results['risk_distribution'], indent=2))

In [ ]:
# Download results (uncomment to download)
# from google.colab import files
# files.download('attrition_results.json')
print("💡 Uncomment the lines above to download the results file")

---

## 📖 Key Terminology Reference

| Term | Definition |
|------|------------|
| **Transformer Encoder** | Neural network using self-attention to learn feature relationships |
| **Self-Attention** | Mechanism that weighs relationships between ALL input features simultaneously |
| **SHAP** | Shapley Additive Explanations — fairly distributes prediction credit to features |
| **LDA** | Latent Dirichlet Allocation — discovers hidden topics in text data |
| **Gibbs Sampling** | Iterative algorithm used by LDA to assign words to topics |
| **Five-Tier Risk** | Low, Early Warning, Moderate, High, Critical classification system |
| **Calibration** | Blending model output with domain heuristics for realistic probabilities |
| **Feature Attribution** | Assigning importance scores to each input variable |
| **SELU** | Self-Normalizing Exponential Linear Unit activation function |
| **Layer Normalization** | Technique to stabilize training by normalizing activations |
| **Binary Cross-Entropy** | Loss function for measuring binary classification error |
| **Xavier Initialization** | Weight initialization strategy preventing gradient issues |

---

## 📚 References

1. Vaswani et al. (2017). *"Attention Is All You Need."* NeurIPS.
2. Lundberg & Lee (2017). *"A Unified Approach to Interpreting Model Predictions."* NeurIPS.
3. Blei, Ng, Jordan (2003). *"Latent Dirichlet Allocation."* JMLR.
4. Baydili & Tasci (2025). *"Predicting Employee Attrition: XAI-Powered Models."* Systems 13, 583.

---

*Notebook created for project review demonstration*